In [ ]:
%reload_ext autoreload
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.neural_network
import sklearn.model_selection
import sklearn.metrics

import tensorflow as tf
from tensorflow import keras

import optuna

In [ ]:
mnist_PATH = '/kaggle/input/digit-recognizer/'

mnist_train = pd.read_csv(mnist_PATH+'train.csv')
mnist_test = pd.read_csv(mnist_PATH+'test.csv')

In [ ]:
X, y = mnist_train.iloc[:,1:].values/255, mnist_train.iloc[:,0].values

X_train, X_validation, y_train, y_validation = sklearn.model_selection.train_test_split(X, y, 
                                                                                        test_size=0.2, 
                                                                                        random_state=0)

In [ ]:
# função objetivo para otimização de hiperparâmetros
def objetivo_rc(trial):
       
    # hiperparâmetros de busca para as Redes Convolucionais
    rc_filters1 = trial.suggest_int("rc_filters1", 32, 256)
    rc_filters2 = trial.suggest_int("rc_filters2", 32, 256)
    rc_padding1 = trial.suggest_categorical("rc_padding1", ["valid", "same"])
    rc_padding2 = trial.suggest_categorical("rc_padding2", ["valid", "same"])
    rc_dropout = trial.suggest_float("rc_dropout", 0.05, 0.30, log = True)
    rc_activation1 = trial.suggest_categorical("rc_activation1", ["relu", "sigmoid", "tanh", "selu"])
    rc_activation2 = trial.suggest_categorical("rc_activation2", ["softmax", "softplus"])
    rc_optimizer = trial.suggest_categorical("rc_optimizer", ["adam", "Nadam", "SGD", "Adamax"])
    rc_batch_size = trial.suggest_int("rc_batch_size", 8, 32)
     
    # Montando a Rede Convolucional
    def m_keras():

        m_keras = keras.models.Sequential()
        m_keras.add(keras.layers.Conv2D(filters=rc_filters1, kernel_size=(3,3), activation="relu", input_shape=(28, 28, 1), padding=rc_padding1))
        m_keras.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = 2))
        m_keras.add(keras.layers.Conv2D(filters=rc_filters2, kernel_size=(3,3), activation="relu", padding=rc_padding2))
        m_keras.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = 1))
        m_keras.add(keras.layers.Flatten())
        m_keras.add(keras.layers.Dense(units=64, activation=rc_activation1))
        m_keras.add(keras.layers.Dropout(rc_dropout))
        m_keras.add(keras.layers.Dense(units=10, activation=rc_activation2))
        m_keras.compile(loss='sparse_categorical_crossentropy', optimizer=rc_optimizer, metrics=['accuracy'])
            
        return m_keras
        
    # wrapper para adequar o modelo keras à interface do sklearn
    m = keras.wrappers.scikit_learn.KerasClassifier(m_keras, epochs = 10, batch_size = rc_batch_size)
    
        
    # montando uma pipeline com pré-processamento+modelo
    #m = sklearn.pipeline.Pipeline([('scaler', sklearn.preprocessing.StandardScaler()), ('rc', rc)])
        
    # retornando acurácia
    acuracias = sklearn.model_selection.cross_val_score(m, X_train.reshape(-1,28,28,1), y_train)
    acuracia = acuracias.mean()
    
    return acuracia

In [ ]:
study_rc = optuna.create_study(direction="maximize")
study_rc.optimize(objetivo_rc, n_trials = 20)

In [ ]:
optuna.visualization.plot_optimization_history(study_rc)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study_rc)

In [ ]:
study_rc.best_params